In [76]:
import itertools
import nltk
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
def eliminate_implication(expression):
    ind = expression.find("->")
    j = ind - 1 
    while expression[j] == '(' or expression[j] == ')' or  (expression[j+1] == ')' and expression[j-1] == '(' ) or expression[j] == ' ':
        j -=1
        
    
    if ind != -1:
        expression = expression[:ind] + "|" + expression[ind+2:]
        expression = expression[:j] + "¬" + expression[j:]
        


        return eliminate_implication(expression)    
    else:
        return expression
    


    
def remove_double_not(expression):
    expression = expression.replace("¬¬", "")
    return expression

words = []

def DeMorgan(expression):
    finish = 0
    cnt = 0
    i = 0
    while i < len(expression):
        if expression[i] =='¬':
            if expression[i + 1] == "(":
                    expression = expression[:i] + expression[i+1:]
                    finish += 1
                    j = i + 1
                    while j < len(expression):
                        if finish == 0 :
                            break
                        if expression[j] == "(":
                            finish += 1
                        elif expression[j] ==")":
                            finish -= 1
                        elif expression[j] == "&":
                            expression = expression[:j] + "|" + expression[j+1:]
                        elif expression[j] == "|":
                            expression = expression[:j]+ "&" + expression[j+1:]
                        elif expression[j] == "¬":
                            expression = expression[:j] + expression[j+1:]
                        elif expression[j] == "P" or expression[j] == "Q" or expression[j] == "M":
                            expression = expression[:j-1] + " ¬" + expression[j:]
                            j += 1

      
                        j += 1
        i += 1

    return expression

    


def standardize_variable_scope(expression):
    x = 0 
    finish = 0 
    for i in range(len(expression)):
        if expression[i] == "∀"or  expression[i] == "∃":
            exp = expression[i+1]

            x = i + 1
            
            while expression[x] != "(":
                x += 1
                
            if expression[x] == '(':
                x+= 1
                finish += 1
                
            
            while x < len(expression):
                if finish == 0:
                    break
                    
                if expression[x] == "(":
                    finish += 1
                elif expression[x] == ")":
                    finish -= 1

                x += 1
            
            while x < len(expression):
                
                if expression[x] == exp:
                    expression = expression[:x] + "z" +expression[x+1:]

                    
                x+=1
            
        
                
            
    return expression
            

def prenex_form(expression):
    quantifiers = ""
    quantified_expression = expression
    i = 0
    while i < len(quantified_expression):
        if quantified_expression[i] in ["∀"]:
            quantifiers += quantified_expression[i]
            j = i + 1
            quantifiers += quantified_expression[i + 1:j + 1]
            quantified_expression = quantified_expression[:i] + quantified_expression[j + 1:]
        i += 1
    return quantifiers + quantified_expression

def skolemization(expression):
    finish = 0
    m = 0
    while expression.find("∃") != -1 :
        i = expression.find("∃")
        if i != -1:
            exp = expression[i+1]
            x = i + 1
            expression = expression[:x-1] + expression[x+2:]
            while expression[x] != "(":
                x += 1

            if expression[x] == '(':
                x+= 1
                finish += 1
            if(expression[x]==exp):
                expression = expression[:x] + "F(z)" +  expression[x+1:]
            else:
                while x < len(expression):
                    if finish == 0 :
                        break
                    if expression[x] == "(":
                        finish += 1
                    elif expression[x] == ")":
                        finish -= 1
                    elif expression[x] == exp:
                        expression = expression[:x] + "F(z)" +  expression[x+1:]

                    x+= 1
            m += 1
            
    return expression

def eliminate_universal_quantifiers(expression):
    exp = ""
    i = 0
    while expression[i] != "(":
        i+= 1
    
    i+=1
    while i < len(expression) - 1:
        exp += expression[i]
        i+= 1
        
    return exp



                

    
def clauses_and_rename_variables(expression):
    clauses = expression.split("&")
    for i in range(len(clauses)):
        for char in clauses[i]:
            if char == "x" or char == "z" or char == "y" or char == "a" or char == "b" or char == "c":
                clauses[i] = clauses[i].replace(char, f'{char}_{i}')
    return clauses
#     print("clau2",clauses)

def Convert_cnf(expression):
    # Step 1: Eliminate Implication
    expression = eliminate_implication(expression)
    print("Step 1 (Eliminate Implication):", expression)
    
    expression = DeMorgan(expression)
    print("Step 2 (Move Negation Inward):", expression)
    
    expression = remove_double_not(expression)
    print("Step 3 (Remove Double Negation):", expression)

    expression = standardize_variable_scope(expression)
    print("Step 4 (Standardize Variable Scope):", expression)

    expression = skolemization(expression)
    print("Step 6 (Skolemization):", expression)

    
    expression = prenex_form(expression)
    print("Step 5 (Prenex Form):", expression)

    
    expression =  eliminate_universal_quantifiers(expression)
    print("Step 7 (Eliminate Universal Quantifiers):", expression)

    expression = clauses_and_rename_variables(expression) 
    print("Step 9 (Clauses and Rename Variables):", expression)

    




# # Example usage
expression = "(∀x ¬(P(x) -> Q(x)) & (∃y ∀x ¬P(x, y))"
Convert_cnf(expression)
expression = "(∀x ¬((Q(x) | M(x))) | (∃x ¬P(x))"
Convert_cnf(expression)
expression = "((∀x (P(x) -> Q(x))))"
Convert_cnf(expression)
expression = "(P(f(a)))"
Convert_cnf(expression)
expression = "(¬(Q(f(a)))"
Convert_cnf(expression)
expression = "(∃x ¬(Q(x) & P(x))"
Convert_cnf(expression)
expression = "((∀x ∃y ∀z ((P(x,y) & P(y,z)) & (Q(z) -> M(z)) & (M(z) -> Q(z)))))"
Convert_cnf(expression)


Step 1 (Eliminate Implication): (∀x ¬(¬P(x) | Q(x)) & (∃y ∀x ¬P(x, y))
Step 2 (Move Negation Inward): (∀x (P(x) & ¬Q(x)) & (∃y ∀x ¬P(x, y))
Step 3 (Remove Double Negation): (∀x (P(x) & ¬Q(x)) & (∃y ∀x ¬P(x, y))
Step 4 (Standardize Variable Scope): (∀x (P(x) & ¬Q(x)) & (∃y ∀z ¬P(z, y))
Step 6 (Skolemization): (∀x (P(x) & ¬Q(x)) & (∀z ¬P(z, F(z)))
Step 5 (Prenex Form): ∀x∀z( (P(x) & ¬Q(x)) & ( ¬P(z, F(z)))
Step 7 (Eliminate Universal Quantifiers):  (P(x) & ¬Q(x)) & ( ¬P(z, F(z))
Step 9 (Clauses and Rename Variables): [' (P(x_0) ', ' ¬Q(x_1)) ', ' ( ¬P(z_2_2, F(z_2_2))']
Step 1 (Eliminate Implication): (∀x ¬((Q(x) | M(x))) | (∃x ¬P(x))
Step 2 (Move Negation Inward): (∀x ( ¬Q(x) & ¬M(x))) | (∃x ¬P(x))
Step 3 (Remove Double Negation): (∀x ( ¬Q(x) & ¬M(x))) | (∃x ¬P(x))
Step 4 (Standardize Variable Scope): (∀x ( ¬Q(x) & ¬M(x))) | (∃z ¬P(z))
Step 6 (Skolemization): (∀x ( ¬Q(x) & ¬M(x))) | (¬P(F(z)))
Step 5 (Prenex Form): ∀x( ( ¬Q(x) & ¬M(x))) | (¬P(F(z)))
Step 7 (Eliminate Universal Quantifie

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
